만약 메시지 list에 의존하지 않고 복잡한 행동을 정의하고 싶다면, state에 filed를 추가해서 해결 할 수 있다.

아래의 예시는 챗봇이 검색 도구를 사용하여 특정 정보(생일과 이름)를 찾아 사용자에게 전달하여 검토하는 시나리오.   

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]
    name: str
    birthday: str

In [3]:
from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId, tool

from langgraph.types import Command, interrupt

@tool
def human_assistance(
    name: str, birthday: str, tool_call_id: Annotated[str, InjectedToolCallId]
) -> str:
    """사용자로부터 도움을 요청"""
    human_response = interrupt(
        {
            "question": "Is this correct?",
            "name": name,
            "birthday": birthday,
        }
    )
    # 만일 정보가 맞다면, 있는 그대로 state를 업데이트
    if human_response.get("correct", "").lower().startswith("y"):
        verified_name = name
        verified_birthday = birthday
        response = "Correct"
    # 맞지 않다면, 사용자의 반응에서 값을 받음
    else:
        verified_name = human_response.get("name", name)
        verified_birthday = human_response.get("birthday", birthday)
        response = f"Made a correction: {human_response}"

    # 예제에서는 tool 안에서 ToolMessage를 사용해서 state를 명시적으로 업데이트
    state_update = {
        "name": verified_name,
        "birthday": verified_birthday,
        "messages": [ToolMessage(response, tool_call_id=tool_call_id)],
    }
    # state를 업데이트하기 위해 tool 안에서 Command를 반환
    return Command(update=state_update)

In [5]:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

tool = TavilySearchResults(max_results=2)
tools = [tool, human_assistance]
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_with_tools = llm.bind_tools(tools)

def chatbot(state:State):
    message = llm_with_tools.invoke(state["messages"])
    assert len(message.tool_calls) <= 1 # 최대 하나의 tool 호출
    return {"messages": [message]}

graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)


In [6]:
user_input = (
    "Can you look up when LangGraph was released? "
    "When you have the answer, use the human_assistance tool for review."
)
config = {"configurable": {"thread_id": "1"}}

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Can you look up when LangGraph was released? When you have the answer, use the human_assistance tool for review.
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_aW8oLKz5SFk2bCMexDjlS1Pv)
 Call ID: call_aW8oLKz5SFk2bCMexDjlS1Pv
  Args:
    query: LangGraph release date
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "langgraph - PyPI", "url": "https://pypi.org/project/langgraph/", "content": "langgraph · PyPI\nSkip to main content Switch to mobile version\n\nSearch PyPI  Search\n\nHelp\nSponsors\nLog in\nRegister\n\nMenu\n\nHelp\nSponsors\nLog in\nRegister\n\nSearch PyPI  Search\nlanggraph 0.2.70\npip install langgraph Copy PIP instructions\nLatest versionReleased: Feb 6, 2025\nBuilding stateful, multi-actor applications with LLMs\nNavigation\n\nProje

In [7]:
human_command = Command(
    resume={
        "name": "LangGraph",
        "birthday": "Jan 17, 2024",
    },
)

events = graph.stream(human_command, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  human_assistance (call_HmKYygLnDHxhXDN3Ew6SpmM5)
 Call ID: call_HmKYygLnDHxhXDN3Ew6SpmM5
  Args:
    name: LangGraph
    birthday: January 22, 2024
================================= Tool Message =================================
Name: human_assistance

Made a correction: {'name': 'LangGraph', 'birthday': 'Jan 17, 2024'}
================================== Ai Message ==================================

LangGraph was officially launched on January 17, 2024.


In [8]:
snapshot = graph.get_state(config)
snapshot.values.items()

dict_items([('messages', [HumanMessage(content='Can you look up when LangGraph was released? When you have the answer, use the human_assistance tool for review.', additional_kwargs={}, response_metadata={}, id='e3f580fd-2b20-45c3-a814-797035d7030c'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_aW8oLKz5SFk2bCMexDjlS1Pv', 'function': {'arguments': '{"query":"LangGraph release date"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 130, 'total_tokens': 152, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac', 'id': 'chatcmpl-BEW2Ds62zHDMRVttCoi9lZE8ex4v2', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-adb2b135-d693-47

In [9]:
{k: v for k, v in snapshot.values.items() if k in {"name", "birthday"}}

{'name': 'LangGraph', 'birthday': 'Jan 17, 2024'}

LangGraph의 state control의 특징

1. 원하는 대로 상태 설계 가능
- 메시지 이외에도 필요한 어떤 데이터든 상태로 관리할 수 있음
- 예: 이름, 생일 등의 정보를 상태에 추가 가능
2. 쉽고 명확한 상태 관리
- 상태 변경을 명확하게 추적하고 제어 가능
- 도구(Tool)를 통해 상태를 체계적으로 업데이트

즉, LangGraph는 개발자가 원하는 방식대로 애플리케이션의 상태를 자유롭게 설계하고 관리할 수 있게 해주는 도구입니다.

In [10]:
graph.update_state(config, {"name": "LangGraph (library)"})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f00887f-7441-6d45-8006-94232e63dba1'}}

In [11]:
snapshot = graph.get_state(config)

{k: v for k, v in snapshot.values.items() if k in ("name", "birthday")}

{'name': 'LangGraph (library)', 'birthday': 'Jan 17, 2024'}